In [1]:
import os 

In [2]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\el papi\\smart-trash-classifier'

In [8]:
from cnnClassifier import logger
from cnnClassifier.pipeline.stage_02_prepare_base_model import PrepareBaseModelTrainingPipeline
import torch
updated_model, model_architecture=PrepareBaseModelTrainingPipeline().main()

model_architecture.load_state_dict(torch.load("artifacts/training/model.pth", map_location="cuda"))


[2025-10-03 00:16:34,130: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-03 00:16:34,133: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-03 00:16:34,134: INFO: common: created directory at: artifacts]
[2025-10-03 00:16:34,136: INFO: common: created directory at: artifacts/prepare_base_model]
[Init] PrepareBaseModel initialized with model: efficientnet_b0
[Pipeline] Preparing base model...
[Step 1] Loading base model...
[Step 2] Freezing base model layers...
[Step 3] Replacing classifier head...
[Done] Base model created.
[Saved] Model saved at: artifacts\prepare_base_model\base_model.pth
[Pipeline Done] Base model prepared and saved.
[Pipeline] Updating base model...
[Step 1] Loading base model...
[Step 2] Freezing base model layers...
[Step 3] Replacing classifier head...
[Done] Base model created.
[Step 1] Loaded base model weights.
[Step 2] Unfreezing classifier...
[Step 3] Unfreezing last 9 backbone layers...
[Saved] Model saved at: ar

<All keys matched successfully>

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [ ]:

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.pth",
            training_data="artifacts/data_ingestion/dataset-resized",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    

In [24]:
import torch
import torch.nn as nn
from pathlib import Path
from cnnClassifier.utils.common import save_json

class Evaluation:
    def __init__(self, config: "EvaluationConfig", model: nn.Module, test_data, device="cuda"):
        
        self.config = config
        self.device = device if torch.cuda.is_available() else "cpu"

        # Initialize model architecture
        self.model = model.to(self.device)

        # Load saved weights
        self.model.load_state_dict(
            torch.load(self.config.path_of_model, map_location=self.device)
        )
        self.model.eval()

        self.test_data = test_data
        self.criterion = nn.CrossEntropyLoss()

    def evaluate(self):
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in self.test_data:
                images, labels = images.to(self.device), labels.to(self.device)

                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                test_loss += loss.item() * images.size(0)

                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        avg_loss = test_loss / total
        accuracy = correct / total * 100
        return {"loss": avg_loss, "accuracy": accuracy}

    def save_results(self, results: dict):
        """Save evaluation results as JSON"""
        save_json(Path("scores.json"), results)


In [25]:
from cnnClassifier.pipeline.stage_02_prepare_base_model import PrepareBaseModelTrainingPipeline
from cnnClassifier.pipeline.stage_03_data_preprocessing import DataPreprocessingTrainingPipeline

prepare_pipeline = PrepareBaseModelTrainingPipeline()
updated_model, model_architecture = prepare_pipeline.main()
data_pipeline = DataPreprocessingTrainingPipeline()
train_loader, val_loader, test_loader = data_pipeline.main()

# config
config = ConfigurationManager()
config_val = config.get_validation_config()

# evaluator (will load model automatically)
evaluator = Evaluation(config=config_val, model=model_architecture, test_data=test_loader)

# evaluate
results = evaluator.evaluate()
print(results)

# save
evaluator.save_results(results)


[2025-10-03 01:23:58,550: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-03 01:23:58,552: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-03 01:23:58,553: INFO: common: created directory at: artifacts]
[2025-10-03 01:23:58,554: INFO: common: created directory at: artifacts/prepare_base_model]
[Init] PrepareBaseModel initialized with model: efficientnet_b0
[Pipeline] Preparing base model...
[Step 1] Loading base model...
[Step 2] Freezing base model layers...
[Step 3] Replacing classifier head...
[Done] Base model created.
[Saved] Model saved at: artifacts\prepare_base_model\base_model.pth
[Pipeline Done] Base model prepared and saved.
[Pipeline] Updating base model...
[Step 1] Loading base model...
[Step 2] Freezing base model layers...
[Step 3] Replacing classifier head...
[Done] Base model created.
[Step 1] Loaded base model weights.
[Step 2] Unfreezing classifier...
[Step 3] Unfreezing last 9 backbone layers...
[Saved] Model saved at: ar